## Compare and analyze TSA predictions 
Start with taking the best predictions with polarity, and score them for binary


In [8]:
import json, os, sys, io, shutil
from pathlib import Path
import tabulate
import pandas as pd
from datetime import date
import zipfile
import tqdm as notebook_tqdm

from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from datasets import load_from_disk, load_dataset
gold_binary = load_dataset("ltg/norec_tsa")
gold_binary = gold_binary["test"]["tsa_tags"]
gold_intense = load_dataset("ltg/norec_tsa", "intensity")
gold_intense = gold_intense["test"]["tsa_tags"]
print(len(gold_binary), len(gold_intense))

1272 1272


In [9]:
preds_path = "/home/egil/gits_wsl/seq-label-github/logs/predictions"
name_filter =  "01191518_tsa-bin_NB-BERT_large_07-b.json" # Best with intensity from 01191518
name_filter =  "lsa-polarity"
# jsons = [f for f in Path(preds_path).iterdir() if f.name.startswith(name_filter)  and  f.name.endswith("json")]
preds1 = json.loads(Path(preds_path, name_filter).read_text())



In [13]:
# Straight forwards binary dataset evaluation
preds = json.loads(Path( "/home/egil/gits_wsl/seq-label-github/logs/predictions/01191518_tsa-bin_NorBERT_3_large_37_b.json").read_text())
golds = gold_binary
assert all([len(p)==len(g)for p,g in zip(preds,golds)]), "Something muffins"

p_tags = sorted(list(set([t for s in preds for t in s ])))
g_tags =   sorted(list(set([t for s in golds for t in s ])))


print("pred",p_tags)
print("gold", g_tags)
print(classification_report(golds, preds, digits=4, zero_division=0))

pred ['B-targ-Negative', 'B-targ-Positive', 'I-targ-Negative', 'I-targ-Positive', 'O']
gold ['B-targ-Negative', 'B-targ-Positive', 'I-targ-Negative', 'I-targ-Positive', 'O']
               precision    recall  f1-score   support

targ-Negative     0.4648    0.3143    0.3750       210
targ-Positive     0.5097    0.6019    0.5520       525

    micro avg     0.5013    0.5197    0.5104       735
    macro avg     0.4872    0.4581    0.4635       735
 weighted avg     0.4969    0.5197    0.5014       735



In [16]:
# gold_intense_path = "/home/egil/gits_wsl/seq-label-github/data/tsa_intensity"
# gold_binary_path = "/home/egil/gits_wsl/seq-label-github/data/tsa_binary"


assert all([len(p)==len(g)for p,g in zip(preds1,gold_binary)]), "Something muffins"

p_tags = sorted(list(set([t[7:] for s in preds1 for t in s ])))
g_tags =   sorted(list(set([t[7:] for s in gold_intense for t in s ])))
print("pred",p_tags)
print("gold", g_tags)
print(classification_report(gold_intense, preds1, digits=4, zero_division=0))

set([t for s in preds1 for t in s ])

pred ['', 'Negative', 'Positive']
gold ['', 'Negative-1', 'Negative-2', 'Negative-3', 'Positive-1', 'Positive-2', 'Positive-3']
                 precision    recall  f1-score   support

  targ-Negative     0.0000    0.0000    0.0000         0
targ-Negative-1     0.0000    0.0000    0.0000        38
targ-Negative-2     0.0000    0.0000    0.0000       126
targ-Negative-3     0.0000    0.0000    0.0000        46
  targ-Positive     0.0000    0.0000    0.0000         0
targ-Positive-1     0.0000    0.0000    0.0000        32
targ-Positive-2     0.0000    0.0000    0.0000       235
targ-Positive-3     0.0000    0.0000    0.0000       258

      micro avg     0.0000    0.0000    0.0000       735
      macro avg     0.0000    0.0000    0.0000       735
   weighted avg     0.0000    0.0000    0.0000       735



{'B-targ-Negative',
 'B-targ-Positive',
 'I-targ-Negative',
 'I-targ-Positive',
 'O'}

In [12]:
pred_bin_from_int = [[t[:-2]  if len(t) > 2 else t for t in s] for s in preds1] 

print(classification_report(gold_binary, pred_bin_from_int, digits=4, zero_division=0))

               precision    recall  f1-score   support

targ-Negative     0.4825    0.3286    0.3909       210
targ-Positive     0.5418    0.5676    0.5544       525

    micro avg     0.5296    0.4993    0.5140       735
    macro avg     0.5122    0.4481    0.4727       735
 weighted avg     0.5249    0.4993    0.5077       735



In [13]:
# Compare with predictions from best binary model
# 01191518_tsa-bin_NB-BERT_large_07.json 
preds2 = json.loads(Path(preds_path, "01191518_tsa-bin_NB-BERT_large_07.json").read_text())
print(classification_report(gold_binary, preds2, digits=4, zero_division=0))

               precision    recall  f1-score   support

targ-Negative     0.4319    0.4381    0.4350       210
targ-Positive     0.5362    0.5790    0.5568       525

    micro avg     0.5077    0.5388    0.5228       735
    macro avg     0.4840    0.5086    0.4959       735
 weighted avg     0.5064    0.5388    0.5220       735

